# Creating and Monitoring a SageMaker Autopilot Experiment using the SageMaker Python SDK

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r s3_bucket_name
s3_bucket_name

In [ ]:
%store -r prefix
prefix

In [ ]:
s3_data_source = \
f"s3://{s3_bucket_name}/{prefix}/training_data.csv"
output_target = f"s3://{s3_bucket_name}/{prefix}/output"
max_candidates = 25

In [ ]:
import sagemaker
session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
from sagemaker.automl.automl import AutoML

experiment = AutoML(
    role=role,
    sagemaker_session=session,
    target_attribute_name="label",
    output_path=output_target,
    max_candidates=25,
    max_runtime_per_training_job_in_seconds=1000,
    total_job_runtime_in_seconds=6000
)

In [ ]:
experiment.fit(inputs=s3_data_source, logs=False, wait=False)

In [ ]:
response = experiment.describe_auto_ml_job()
response

In [ ]:
from pprint import pprint
from time import sleep

In [ ]:
%%time

status = "InProgress"

while status == "InProgress":
    response = experiment.describe_auto_ml_job()
    status = response['AutoMLJobStatus']
    secondary_status = response['AutoMLJobSecondaryStatus']
    
    print(f"{status} - {secondary_status}")

    sleep(15)

In [ ]:
response = experiment.describe_auto_ml_job()

In [ ]:
delta = response['EndTime'] - response['CreationTime']
total_minutes = int(delta.total_seconds() / 60)
total_minutes

In [ ]:
artifacts = response['AutoMLJobArtifacts']
artifacts

In [ ]:
for s3_path in list(artifacts.values()):
    !aws s3 cp {s3_path} tmp/.


In [ ]:
best = experiment.best_candidate()
best

In [ ]:
best['FinalAutoMLJobObjectiveMetric']

In [ ]:
autopilot_job_name = response['AutoMLJobName']
%store autopilot_job_name
autopilot_job_name